# EECE 5552 Lab 2: Multi-Modal Control of the UR12e: Simulation, Pendant, and Python

 This lab focuses on controlling and simulating UR12e robotic movements. We will practice basic control of the UR12e, use PolyScope on URSim to simulate these movements, and then implement them on the real robot arm. We will also cover advanced path planning using ROS2 Jazzy and MoveIt2 to explore complex trajectory planning techniques.

### After completing this lab, a well-prepared student should be able to:  

1. Safely operate a UR12e manipulator
	- Power on, initialize, and shut down the robot
	- Respond correctly to emergency stops and protective stops
2. Program basic motions in PolyScope
	- Create joint-space and Cartesian-space motions
	- Define waypoints and adjust speed and acceleration
	- Execute programs in both URSim and on the physical robot
3. Transfer programs from simulation to hardware
	- Validate trajectories in URSim
	- Deploy programs to the real UR12e while accounting for joint limits, payload, and singularities
4. Set up a ROS2 Jazzy control pipeline
	- Launch required ROS2 nodes
	- Connect to the UR12e or simulator
	- Verify state feedback and command execution
6. Use MoveIt for motion planning
	- Configure the planning scene
	- Define start and goal poses
	- Generate collision-aware trajectories

### Key Concepts

Key terms you will encounter in this lab:

- [UR12e](https://www.universal-robots.com/products/ur12e/): a medium duty industrial collaborative robot that combines both long reach and high payload. It's our robot arm used for automation tasks, capable of precise movements and programmable control.

- [URSim](https://www.universal-robots.com/download/software-ur-series/simulator-non-linux/offline-simulator-ur-series-e-series-ur-sim-for-non-linux-5222/): Simulation software that mimics the UR12e’s real-world behavior, allowing for safe development and testing without using the physical robot.

- [URScript](./docs/script_directory_Poly5_22.pdf): A scripting language used to control the UR5, either through manual commands or automation within the PolyScope interface.

- [URControl](https://www.universal-robots.com/manuals/EN/HTML/SW10_6/Content/prod-scriptmanual/G5/Connecting_to_URControl.htm) is the low-level robot controller running on the Embedded PC in the Control Box cabinet. When the PC boots up, the URControl starts up as a daemon (i.e., a service) and the PolyScope or Graphical User Interface connects as a client using a local TCP/IP connection.

- [Teach Pendant](https://www.universal-robots.com/marketplace/products/01tP4000006vfFtIAI/): an example of a Teach Pendant with 3-Position Enabling Device.  

- [PolyScope](https://www.universal-robots.com/products/polyscope-5/): The UR12e’s user-friendly graphical interface for programming, which automatically generates URScript code in the background.

- [ROS2 Jazzy](https://docs.ros.org/en/jazzy/index.html): An open-source middleware that enables advanced control and integration of the UR12e with external components, allowing real-time performance and complex tasks like path planning using algorithms such as Rapidly-exploring Random Tree (RRT).

- [RViz2](https://docs.ros.org/en/jazzy/Tutorials/Intermediate/RViz/RViz-User-Guide/RViz-User-Guide.html): a 3D visualizer for the ROS2 framework.

- [MoveIt2](https://github.com/moveit/moveit2): A powerful software framework for motion planning, manipulation, and control of robots. It integrates with ROS to provide tools for collision detection, kinematics, and trajectory planning, helping to automate complex movements for robotic arms like the UR12e.

### Collaboration statement: 
You should complete this assignment without collaboration with other students in the course or elsewhere. Please ask for help from the TAs or course instructors if you are getting stuck!

## Part 1 - Simulation

### Step 1: Install A Virtual Machine and Run URSim

URSim is the simulator developed by Universal Robots (UR) for programming and testing their robotic arms, including the UR12e, without needing the physical robot. 

- URSim allows you to simulate the robot's movements and behavior in a virtual environment;

- URSim allows you to test code, create programs, and debug without the risks or constraints of working directly with UR12e.

The URSim is basically a duplicate of the PolyScope app on the Teach Pendant and runs on a Linux simulation environment. 

#### Choose a Virtual Machine to use

- *Recommend* [VirtualBox](https://www.virtualbox.org/)

#### Download an Offline Simulator 

- Case 1: x86-64 (tested)

    If you have a computer running Windows on x86-64 or running MacOS on Intel chips, use the following links to get a version (*recommend version **5.22.2**, since it matches the version used on our UR12e.*):

    - [Offline Simulator - e-Series - UR Sim for non Linux 5.12.6 LTS](https://www.universal-robots.com/download/software-e-series/simulator-non-linux/offline-simulator-e-series-ur-sim-for-non-linux-5126-lts/) 
    - [Offline Simulator - UR-Series & e-Series - UR Sim for non Linux 5.22.2](https://www.universal-robots.com/download/software-ur-series/simulator-non-linux/offline-simulator-ur-series-e-series-ur-sim-for-non-linux-5222/)

- Case 2: ARM (*not tested*)

    If you have a computer running Windows on ARM, or running MacOS on Apple Silicons, get online resources by yourself.


- Case 3: Linux (*not tested*)

    If you have a Linux computer running like Ubuntu, use the following link to get a version (*recommend version **5.22.2**, since it matches the version used on our UR12e.*):

    - [Offline Simulator - UR-Series & e-Series - UR Sim for Linux 5.22.2](https://www.universal-robots.com/download/software-ur-series/simulator-linux/offline-simulator-ur-series-e-series-ur-sim-for-linux-5222/)

    - [Offline Simulator - e-Series - UR Sim for Linux 5.12.6 LTS](https://www.universal-robots.com/download/software-e-series/simulator-linux/offline-simulator-e-series-ur-sim-for-linux-5126-lts/) 

For the previous step, you need to login first to download files. You can also directly get from the shared OneDrive folder. Check [the folder link on Canvas](https://northeastern-my.sharepoint.com/:f:/r/personal/x_li1_northeastern_edu/Documents/EECE%205552%20Spring%202026/Essential%20Software/URSim?csf=1&web=1&e=MgRq4u).

#### Run the simulator in virtual machine

*Use URSim on VirtualBox in Windows as an example*

1. Extract the rar file (URSim) first; 

2. Open VirtualBox app and click the green cross symbol as "Open", to choose the simulation file in the extracted URSim folder.

3. Before open the URSim app on VirtualBox, click the image then open Settings and change the Network to Bridge mode. It's for exposing your URSim to a regular network that python scripts can access.

4. Open the URSim.

Check [a guideline](./docs/Guideline_Install_VirtualBox_and_URSim_on_your_Windows_pc.pdf) which was made for an older version of URSim for similar setups.


### Step 2: Get familiar with PolyScope 5

Check Chapter 8 - First Time Use in [UR12e User Manual](./docs/UR12e_User_Manual_en_Global.pdf):

- 8.2.2 Move Robot into Position (Page 111). Use this to move robot to Home position whenever initialize or reset positions. 
    - 8.4.7 Safe Home Position (Page 144). Safe Home is a return position defined by using the user-defined Home Position.

- 8.2.7 Basic Program Nodes: Move (Page 117). 
    - The Move command allows the robot to move from point A to point B.
- 8.2.8 Basic Program Nodes: Waypoints (Page 120). 
    - Waypoints are one of the most central parts of a robot program, telling the robot armwhere to go one movement at a time.
- 14.4 Program and Installation Manager (Page 196). 
    - Learn how to create, save, and open saved program files.

### Step 3: Complete the following tasks

Create a new program to perform a pick-and-place task using the UR12e in URSim:

- At least 5 waypoints:
    - **Waypoint 1** as your defined Home Position;
    - **Waypoint 2** is the position where the robot arm moves from the Home position to Waypoint 3. Choose a position above the Pick location;
    - **Waypoint 3** is the position where the Pick location is;
    - **Waypoint 4** is the position where the robot arm moves from Waypoint 3 to the Place location. Choose a position above the Place location;
    - **Waypoint 5** is the position where the Place location is;
- Move the robot arm in a order as:
    - *Waypoint 1 -> Waypoint 2 -> Waypoint 3 -> Waypoint 2 -> Waypoint 1 -> Waypoint 4 -> Waypoint 5 -> Waypoint 4 -> Waypoint 1*
- Adjust speed or acceleration as needed to make the movement slow and smooth.

- Ensure the trajectory is safe to execute and free of any potential collisions, such as contact with the table surface.

Save the program file as ***FirstName_LastNameInitial_PnP.urp***.

**Record a video** of your robot arm running in URSim. Use your preferred screen recording software to get a clear video with .mp4 format.

## Part 2: Teach Pendant

### Step 4: Save and export the .urp prgram file

Follow [this guideline](./docs/Guideline_Transferring_Files_between_URSim_on_VirtualBox_and_PC_using_SCP.pdf) to transfer your save .urp file to your computer.

Export the program to a flash drive formatted as FAT or FAT32; otherwise, the Teach Pendant will not be able to read it. 

### Step 5: Run your .urp program file on the Teach Pendant

Plug your flash drive into the Teach Pendant (*USB port is on the top right corner*), then read the program file. 

Simulate your program first, then run it. Check if the robot hits any joint limits or singularities. Check Page 26 on [UR12e User Manual](./docs/UR12e_User_Manual_en_Global.pdf) for the details of simulation mode.

**Record a video** of the UR12e moving with your smartphone and export the video as .mp4.

**Save your .urp program file** after you are satisfied with the motion. Basically it needs to fit the safety requirements on the real UR12e robot.

## Part 3: Python

### Step 6: Get real-time status of UR12e via Python and RTDE

1. Install the RTDE library by following guideline in this link:
[RTDE_Python_Client_Library](https://github.com/UniversalRobots/RTDE_Python_Client_Library/tree/main)

    To make it simple, just use the following command lines. Recommend to run it in a Python virtual environment. Check [here for help on Python Virtual Environment](https://docs.python.org/3/library/venv.html).

```bash
#!/usr/bin/env bash
git clone https://github.com/UniversalRobots/RTDE_Python_Client_Library
pip install ./RTDE_Python_Client_Library


2. Then get the IP address of UR12e in your URSim by following [this guideline](./docs/Guideline_Transferring_Files_between_URSim_on_VirtualBox_and_PC_using_SCP.pdf).

3. Run the script: [control_ur12e_via_python_GetStatus.py](./src/control_ur12e_via_python_GetStatus.py), with your robot's IP address.

4. You should get the following output [snapshot here](./pics/Screenshot_status_via_python.png).

**Take a snapshot** of your terminal to show the live status of robot. 

### Step 7: Control UR12e via Python and URScript

We are controlling robot via an UR scripting designed langauge called ***URScript***. In following command, it's basically controlling the 6 joints to move to the designated angles. Check the [details on MoveJ or other commands in URScript in this manual](./docs/script_directory_Poly5_22.pdf).

Change the variables in the **movej** command to control the robot arm.

The Python methods in Steps 6 and 7 work for both URSim and the real robot. The only required change is the IP address. When controlling a real robot without internet access, configure your computer to be on the same local network as the robot.

1. With your robot's IP, run this script: [control_ur12e_via_python_GoHome.py](./src/control_ur12e_via_python_GoHome.py). 
2. You should see the robot arm moving by following your commands in the scripts.

**Take a video** of the robot arm (either in URSim or physical UR12e robot arm) moves to those two home positions.

a = acceleration (unit?)

v = velocity (unit?)

```python
# Home - 1 
movej([0, -1.57, 1.57, -1.57, 1.57, 0], a=1.2, v=0.25)

# Home - 2, as the UR12e's default Home Position
movej([0, -1.57, 0, -1.57, 0, 0], a=1.2, v=0.25)

### Step 8: Get real-time status of UR12e via ROS2 Jazzy and RViz2

For first-time use, you need to install the ros2 jazzy UR library to connect UR12e via ROS2 Jazzy and check live status via RViz2.

Follow the steps in the following tutorial to install the libraries:

- https://docs.ros.org/en/jazzy/p/ur_robot_driver/doc/installation/installation.html


Follow steps in this tutorial to run RViz2 to check status of your robot: 

- https://docs.ros.org/en/jazzy/p/ur_robot_driver/doc/usage/startup.html 

**Take a snapshot** of your RViz window with the live status from scripts in Step 6.

### Step 9: Simple path-planning via ROS2 Jazzy and MoveIt2

Follow steps in this following tutorial to run MoveIt2 to control UR12e:

- https://docs.universal-robots.com/Universal_Robots_ROS2_Documentation/doc/ur_robot_driver/ur_robot_driver/doc/usage/move.html#using-moveit

Basically, you need to open two terminals:

```bash
#!/usr/bin/env bash
ros2 launch ur_robot_driver ur_control.launch.py ur_type:=ur12e robot_ip:=0.0.0.0

```bash
#!/usr/bin/env bash
ros2 launch ur_moveit_config ur_moveit.launch.py ur_type:=ur12e launch_rviz:=true

Change the goal on RViz2 and click [Plan] and [Plan and Excute]. Before you test it on the physical UR12e, make sure the planned path is safe to perform.

**Make a video** of UR12e moving at least from one position to anther. Use this [demo video](https://northeastern-my.sharepoint.com/:f:/r/personal/x_li1_northeastern_edu/Documents/EECE%205552%20Spring%202026/UR12e%20Robot%20Arm?csf=1&web=1&e=rajuxh) as a reference.

## What to submit for Lab 2:

### Submission List

1. Video from Step 3, name it as **Video_URSim.mp4**;
2. Video from Step 5, name it as **Video_TeachPendant**
3. Your .URP file from Step 5, name it as ***FirstName_LastNameInitial_PnP.urp***
4. Snapshot from Step 6, name it as **Snapshot_LiveStatus_Python.png**
5. Video from Step 7, name it as **Video_URScript.mp4**
6. Snapshot from Step 8, name it as **Snapshot_RViz2.png**
7. Video from Step 9, name it as **Video_MoveIt2.mp4** 

### Submission Structure

Your GitHub repository should be accessible through "https: // github.com/ your username/EECE5552" 

Your GitHub repository should look like this structure when cloned locally:

<pre>
EECE5552
├── lab2
│   ├── snapshots
│   │   ├── Snapshot_LiveStatus_Python.png
│   │   └── Snapshot_RViz2.png
│   ├── videos
│   │   ├── Video_URSim.mp4
│   │   ├── Video_TeachPendant.mp4
│   │   ├── Video_URScript.mp4
│   │   └── Video_MoveIt2.mp4
│   ├── README.md (brief explanation of what you have done)
│   └── src
│       └── FirstName_LastNameInitial_PnP.urp 
└── README.md (update the section for Lab 2)
</pre>


### Assessment

Total: 100 points
- Snapshots: 10 points * 2
- Videos: 15 points * 4
- Program file: 20 points * 1
- File Structure: If your github repo doesn't follow the required structure, up to 20 points will be taken off.
